<a href="https://colab.research.google.com/github/Articbug/Telecom-CDR-Analytics-Platform/blob/main/Notebooks/6_CDR_Security_RBAC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
#   CDR SECURITY - RBAC + DATA MASKING
#   Wipro CDR Analytics Project
#   Author: Chandan Sahoo, Bismaya Ranjan Sahoo, Debasish Sahoo
# ============================================================

import subprocess, sys

packages = ['snowflake-connector-python', 'pandas']
for package in packages:
    subprocess.run(
        [sys.executable, '-m', 'pip', 'install', package, '--quiet', '--disable-pip-version-check'],
        capture_output=True
    )

print('All libraries installed successfully!')

import snowflake.connector
import pandas as pd

def get_connection():
    return snowflake.connector.connect(
        account  = 'bopsoxz-lr52214',
        user     = 'CHANDANSAHOO',
        password = 'Chandansahoosnowflake5',
        database = 'TELECOM_DWH',
        schema   = 'DWH',
        warehouse= 'TRANSFORM_WH'
    )

conn   = get_connection()
cursor = conn.cursor()
cursor.execute('SELECT CURRENT_USER(), CURRENT_DATABASE(), CURRENT_WAREHOUSE()')
row = cursor.fetchone()
print(f'Connected successfully!')
print(f'   User:      {row[0]}')
print(f'   Database:  {row[1]}')
print(f'   Warehouse: {row[2]}')
conn.close()

All libraries installed successfully!
Connected successfully!
   User:      CHANDANSAHOO
   Database:  TELECOM_DWH
   Warehouse: TRANSFORM_WH


In [ ]:
# ============================================================
#   CELL 2: RBAC - ROLES AND PRIVILEGES
# ============================================================
print('=' * 55)
print('   RBAC - ROLES AND PRIVILEGES')
print('=' * 55)

conn   = get_connection()
cursor = conn.cursor()

# ── 1. CREATE ROLES
print('\n1. Creating Roles:')
for role in ['CDR_ADMIN', 'CDR_ANALYST', 'CDR_VIEWER']:
    cursor.execute(f'CREATE ROLE IF NOT EXISTS {role}')
    print(f'   Role created: {role}')

# ── 2. GRANT PRIVILEGES
print('\n2. Granting Privileges:')

# CDR_ADMIN - full access
cursor.execute('GRANT ALL PRIVILEGES ON DATABASE TELECOM_DWH TO ROLE CDR_ADMIN')
cursor.execute('GRANT ALL PRIVILEGES ON ALL SCHEMAS IN DATABASE TELECOM_DWH TO ROLE CDR_ADMIN')
cursor.execute('GRANT ALL PRIVILEGES ON ALL TABLES IN DATABASE TELECOM_DWH TO ROLE CDR_ADMIN')
print('   CDR_ADMIN  -> Full access (ALL privileges)')

# CDR_ANALYST - read + write staging
cursor.execute('GRANT USAGE ON DATABASE TELECOM_DWH TO ROLE CDR_ANALYST')
cursor.execute('GRANT USAGE ON SCHEMA TELECOM_DWH.STAGING TO ROLE CDR_ANALYST')
cursor.execute('GRANT USAGE ON SCHEMA TELECOM_DWH.DWH TO ROLE CDR_ANALYST')
cursor.execute('GRANT SELECT, INSERT ON ALL TABLES IN SCHEMA TELECOM_DWH.STAGING TO ROLE CDR_ANALYST')
cursor.execute('GRANT SELECT ON ALL TABLES IN SCHEMA TELECOM_DWH.DWH TO ROLE CDR_ANALYST')
print('   CDR_ANALYST -> Read + Write STAGING, Read DWH')

# CDR_VIEWER - read only
cursor.execute('GRANT USAGE ON DATABASE TELECOM_DWH TO ROLE CDR_VIEWER')
cursor.execute('GRANT USAGE ON SCHEMA TELECOM_DWH.DWH TO ROLE CDR_VIEWER')
cursor.execute('GRANT SELECT ON ALL TABLES IN SCHEMA TELECOM_DWH.DWH TO ROLE CDR_VIEWER')
print('   CDR_VIEWER  -> Read only DWH')

# ── 3. VERIFY ROLES
print('\n3. Verifying Roles:')
cursor.execute("SHOW ROLES LIKE 'CDR%'")
rows     = cursor.fetchall()
cols     = [d[0] for d in cursor.description]
df_roles = pd.DataFrame(rows, columns=cols)
print(df_roles[['created_on', 'name', 'owner']].to_string(index=False))

cursor.close()
conn.close()
print('\nRBAC Complete!')

   RBAC - ROLES AND PRIVILEGES

1. Creating Roles:
   Role created: CDR_ADMIN
   Role created: CDR_ANALYST
   Role created: CDR_VIEWER

2. Granting Privileges:
   CDR_ADMIN  -> Full access (ALL privileges)
   CDR_ANALYST -> Read + Write STAGING, Read DWH
   CDR_VIEWER  -> Read only DWH

3. Verifying Roles:
                      created_on        name        owner
2026-02-22 07:08:16.940000-08:00   CDR_ADMIN ACCOUNTADMIN
2026-02-22 07:08:17.243000-08:00 CDR_ANALYST ACCOUNTADMIN
2026-02-22 07:08:17.532000-08:00  CDR_VIEWER ACCOUNTADMIN

RBAC Complete!


In [ ]:
# ============================================================
#   CELL 3: DATA MASKING
# ============================================================
print('=' * 55)
print('   DATA MASKING POLICY')
print('=' * 55)

conn   = get_connection()
cursor = conn.cursor()

# ── 1. CREATE MASKING POLICY
print('\n1. Creating Masking Policies:')

# Phone number masking
cursor.execute('''
    CREATE OR REPLACE MASKING POLICY TELECOM_DWH.DWH.MASK_PHONE_NUMBER
    AS (VAL VARCHAR) RETURNS VARCHAR ->
    CASE
        WHEN CURRENT_ROLE() IN ('CDR_ADMIN', 'ACCOUNTADMIN') THEN VAL
        ELSE CONCAT(LEFT(VAL, 4), '********', RIGHT(VAL, 2))
    END
''')
print('   Phone masking policy created')
print('   CDR_ADMIN/ACCOUNTADMIN -> full number visible')
print('   CDR_ANALYST/CDR_VIEWER -> 91XX********XX')

# Charge amount masking
cursor.execute('''
    CREATE OR REPLACE MASKING POLICY TELECOM_DWH.DWH.MASK_CHARGE_AMOUNT
    AS (VAL NUMBER) RETURNS NUMBER ->
    CASE
        WHEN CURRENT_ROLE() IN ('CDR_ADMIN', 'ACCOUNTADMIN') THEN VAL
        ELSE NULL
    END
''')
print('   Charge amount masking policy created')
print('   CDR_ADMIN/ACCOUNTADMIN -> amount visible')
print('   CDR_ANALYST/CDR_VIEWER -> NULL')

# ── 2. APPLY MASKING POLICIES
print('\n2. Applying Masking Policies:')
cursor.execute('''
    ALTER TABLE TELECOM_DWH.STAGING.STG_CDR
    MODIFY COLUMN CALLING_NUMBER
    SET MASKING POLICY TELECOM_DWH.DWH.MASK_PHONE_NUMBER
''')
print('   Masking applied to STG_CDR.CALLING_NUMBER')

cursor.execute('''
    ALTER TABLE TELECOM_DWH.STAGING.STG_CDR
    MODIFY COLUMN CALLED_NUMBER
    SET MASKING POLICY TELECOM_DWH.DWH.MASK_PHONE_NUMBER
''')
print('   Masking applied to STG_CDR.CALLED_NUMBER')

cursor.execute('''
    ALTER TABLE TELECOM_DWH.STAGING.STG_CDR
    MODIFY COLUMN CHARGE_AMOUNT
    SET MASKING POLICY TELECOM_DWH.DWH.MASK_CHARGE_AMOUNT
''')
print('   Masking applied to STG_CDR.CHARGE_AMOUNT')

# ── 3. VERIFY MASKING
print('\n3. Verifying Data Masking (ACCOUNTADMIN sees full data):')
cursor.execute('''
    SELECT CALLING_NUMBER, CALLED_NUMBER, CALL_TYPE, CHARGE_AMOUNT
    FROM TELECOM_DWH.STAGING.STG_CDR
    LIMIT 5
''')
rows    = cursor.fetchall()
cols    = [d[0] for d in cursor.description]
df_mask = pd.DataFrame(rows, columns=cols)
print(df_mask.to_string(index=False))

# ── 4. SHOW ALL MASKING POLICIES
print('\n4. All Masking Policies:')
cursor.execute('SHOW MASKING POLICIES IN DATABASE TELECOM_DWH')
rows    = cursor.fetchall()
cols    = [d[0] for d in cursor.description]
df_pol  = pd.DataFrame(rows, columns=cols)
print(df_pol[['created_on', 'name', 'schema_name', 'owner']].to_string(index=False))

cursor.close()
conn.close()
print('\nData Masking Complete!')
print('=' * 55)

   DATA MASKING POLICY

1. Creating Masking Policies:
   Phone masking policy created
   CDR_ADMIN/ACCOUNTADMIN -> full number visible
   CDR_ANALYST/CDR_VIEWER -> 91XX********XX
   Charge amount masking policy created
   CDR_ADMIN/ACCOUNTADMIN -> amount visible
   CDR_ANALYST/CDR_VIEWER -> NULL

2. Applying Masking Policies:
   Masking applied to STG_CDR.CALLING_NUMBER
   Masking applied to STG_CDR.CALLED_NUMBER
   Masking applied to STG_CDR.CHARGE_AMOUNT

3. Verifying Data Masking (ACCOUNTADMIN sees full data):
CALLING_NUMBER CALLED_NUMBER CALL_TYPE CHARGE_AMOUNT
  918689968863  916440173413       SMS        0.1000
  918453112268  916663640499     VOICE        0.7000
  916438859907  919654383668     VOICE        4.5083
  916431198492  916420814950      DATA        0.4458
  919768614462  917437365878     VOICE        0.2500

4. All Masking Policies:
                      created_on               name schema_name        owner
2026-02-22 07:09:29.508000-08:00 MASK_CHARGE_AMOUNT         

SyntaxError: incomplete input (ipython-input-1277287299.py, line 87)

In [ ]:
# ============================================================
#   CELL 4: DATA GOVERNANCE
# ============================================================
print('=' * 55)
print('   DATA GOVERNANCE')
print('=' * 55)

conn   = get_connection()
cursor = conn.cursor()

# ── 1. ROW ACCESS POLICY
print('\n1. Row Access Policy:')
cursor.execute(
    "CREATE OR REPLACE ROW ACCESS POLICY TELECOM_DWH.DWH.CDR_ROW_ACCESS "
    "AS (network_type VARCHAR) RETURNS BOOLEAN -> "
    "CASE "
    "WHEN CURRENT_ROLE() = 'CDR_ADMIN'   THEN TRUE "
    "WHEN CURRENT_ROLE() = 'CDR_ANALYST' THEN network_type IN ('4G', '5G') "
    "WHEN CURRENT_ROLE() = 'CDR_VIEWER'  THEN network_type = '4G' "
    "ELSE FALSE END"
)
print('   Row Access Policy created: CDR_ROW_ACCESS')
print('   CDR_ADMIN   -> ALL network types')
print('   CDR_ANALYST -> 4G and 5G only')
print('   CDR_VIEWER  -> 4G only')

cursor.execute(
    "ALTER TABLE TELECOM_DWH.STAGING.STG_CDR "
    "ADD ROW ACCESS POLICY TELECOM_DWH.DWH.CDR_ROW_ACCESS ON (NETWORK_TYPE)"
)
print('   Row Access Policy applied to STG_CDR')

# ── 2. DATA CLASSIFICATION TAGS
print('\n2. Data Classification Tags:')
cursor.execute(
    "CREATE TAG IF NOT EXISTS TELECOM_DWH.DWH.DATA_SENSITIVITY "
    "ALLOWED_VALUES 'PUBLIC', 'INTERNAL', 'CONFIDENTIAL', 'RESTRICTED'"
)
print('   Tag created: DATA_SENSITIVITY')

cursor.execute("ALTER TABLE TELECOM_DWH.STAGING.STG_CDR MODIFY COLUMN CALLING_NUMBER SET TAG TELECOM_DWH.DWH.DATA_SENSITIVITY = 'CONFIDENTIAL'")
cursor.execute("ALTER TABLE TELECOM_DWH.STAGING.STG_CDR MODIFY COLUMN CALLED_NUMBER  SET TAG TELECOM_DWH.DWH.DATA_SENSITIVITY = 'CONFIDENTIAL'")
cursor.execute("ALTER TABLE TELECOM_DWH.STAGING.STG_CDR MODIFY COLUMN CHARGE_AMOUNT  SET TAG TELECOM_DWH.DWH.DATA_SENSITIVITY = 'RESTRICTED'")
cursor.execute("ALTER TABLE TELECOM_DWH.STAGING.STG_CDR MODIFY COLUMN IS_FRAUD       SET TAG TELECOM_DWH.DWH.DATA_SENSITIVITY = 'RESTRICTED'")
print('   CALLING_NUMBER -> CONFIDENTIAL')
print('   CALLED_NUMBER  -> CONFIDENTIAL')
print('   CHARGE_AMOUNT  -> RESTRICTED')
print('   IS_FRAUD       -> RESTRICTED')

# ── 3. AUDIT LOG
print('\n3. Audit Logging:')
cursor.execute(
    "CREATE TABLE IF NOT EXISTS TELECOM_DWH.DWH.AUDIT_LOG ("
    "LOG_ID           NUMBER AUTOINCREMENT PRIMARY KEY, "
    "EVENT_TIME       TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP(), "
    "USER_NAME        VARCHAR(100), "
    "ROLE_NAME        VARCHAR(100), "
    "TABLE_NAME       VARCHAR(100), "
    "ACTION           VARCHAR(50), "
    "RECORDS_AFFECTED NUMBER, "
    "STATUS           VARCHAR(20))"
)
print('   Audit log table created')

cursor.execute(
    "INSERT INTO TELECOM_DWH.DWH.AUDIT_LOG "
    "(USER_NAME, ROLE_NAME, TABLE_NAME, ACTION, RECORDS_AFFECTED, STATUS) VALUES "
    "('CHANDANSAHOO', 'ACCOUNTADMIN', 'STG_CDR',  'SELECT',  50000, 'SUCCESS'),"
    "('CHANDANSAHOO', 'ACCOUNTADMIN', 'FACT_CDR', 'INSERT',  50000, 'SUCCESS'),"
    "('CHANDANSAHOO', 'ACCOUNTADMIN', 'FACT_CDR', 'DELETE',  138,   'SUCCESS'),"
    "('CHANDANSAHOO', 'ACCOUNTADMIN', 'FACT_CDR', 'RECOVER', 138,   'SUCCESS'),"
    "('CHANDANSAHOO', 'CDR_ANALYST',  'STG_CDR',  'SELECT',  1000,  'SUCCESS'),"
    "('CHANDANSAHOO', 'CDR_VIEWER',   'FACT_CDR', 'SELECT',  500,   'SUCCESS')"
)
print('   Audit log entries inserted')

cursor.execute(
    "SELECT USER_NAME, ROLE_NAME, TABLE_NAME, ACTION, RECORDS_AFFECTED, STATUS "
    "FROM TELECOM_DWH.DWH.AUDIT_LOG ORDER BY LOG_ID"
)
rows     = cursor.fetchall()
cols     = [d[0] for d in cursor.description]
df_audit = pd.DataFrame(rows, columns=cols)
print('\n   Audit Log:')
print(df_audit.to_string(index=False))

conn.commit()
cursor.close()
conn.close()
print('\nData Governance Complete!')
print('=' * 55)

   DATA GOVERNANCE

1. Row Access Policy:
   Row Access Policy created: CDR_ROW_ACCESS
   CDR_ADMIN   -> ALL network types
   CDR_ANALYST -> 4G and 5G only
   CDR_VIEWER  -> 4G only
   Row Access Policy applied to STG_CDR

2. Data Classification Tags:
   Tag created: DATA_SENSITIVITY
   CALLING_NUMBER -> CONFIDENTIAL
   CALLED_NUMBER  -> CONFIDENTIAL
   CHARGE_AMOUNT  -> RESTRICTED
   IS_FRAUD       -> RESTRICTED

3. Audit Logging:
   Audit log table created
   Audit log entries inserted

   Audit Log:
   USER_NAME    ROLE_NAME TABLE_NAME  ACTION  RECORDS_AFFECTED  STATUS
CHANDANSAHOO ACCOUNTADMIN    STG_CDR  SELECT             50000 SUCCESS
CHANDANSAHOO ACCOUNTADMIN   FACT_CDR  INSERT             50000 SUCCESS
CHANDANSAHOO ACCOUNTADMIN   FACT_CDR  DELETE               138 SUCCESS
CHANDANSAHOO ACCOUNTADMIN   FACT_CDR RECOVER               138 SUCCESS
CHANDANSAHOO  CDR_ANALYST    STG_CDR  SELECT              1000 SUCCESS
CHANDANSAHOO   CDR_VIEWER   FACT_CDR  SELECT               500 SUC